In [1]:
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('C:/ZhangLI/Codes/DataSet/新闻文本分类/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
#train_df[['text','label_ft']].iloc[:-5000].to_csv('train.csv', index=None, header=None, sep='\t')


In [3]:
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df.head()

,label,text,label_ft
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...,__label__2
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...,__label__11
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...,__label__3
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...,__label__2
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...,__label__3


In [4]:
import fasttext

In [ ]:

model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=25, loss="hs")

# 至少要搞清楚 输入输出 的形式，预训练或者做预测的形式

In [ ]:
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))
# 0.82

In [5]:
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# 使用测试工作 测试fasttext
# https://fasttext.cc/docs/en/unsupervised-tutorial.html
import fasttext
model = fasttext.train_supervised(input="C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.train")

In [7]:
model.save_model('model_cooking.bin')

In [20]:
model.predict("Which baking dish is best to bake a banana bread ?")

(('__label__baking',), array([0.06630655]))

In [26]:
model.predict("Why not put knives in the dishwasher?")
model.predict("Why not put knives in the dishwasher?", k=5)  # 获取前5个标签

(('__label__baking',
  '__label__food-safety',
  '__label__bread',
  '__label__substitutions',
  '__label__equipment'),
 array([0.06427463, 0.06387777, 0.03709028, 0.03550518, 0.03373712]))

In [25]:
model.test("C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.valid")

(3000, 0.14533333333333334, 0.06285137667579645)

In [24]:
model.test("C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.valid", k=5)

(3000, 0.06606666666666666, 0.14285714285714285)

In [29]:
# 进行一个调参
import fasttext
model = fasttext.train_supervised(input="C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.train.new")
model.test("C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.valid.new")

(3000, 0.16766666666666666, 0.07250973043102206)

In [30]:
# 增加epoch， 减小学习率
model = fasttext.train_supervised(input="C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.train.new", epoch=25, lr=0.1)
model.test("C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.valid.new")

(3000, 0.5173333333333333, 0.22372783624044976)

In [32]:
# 增加epoch， 减小学习率
model = fasttext.train_supervised(input="C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.train.new", epoch=25, lr=0.1, wordNgrams=2, bucket=200000, dim=50, loss='hs')
model.test("C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.valid.new")

(3000, 0.43433333333333335, 0.18783335735908893)

In [38]:
# 可以指定多个多个二分，实现多分类概率
model = fasttext.train_supervised(input="C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.train.new", epoch=25, lr=0.1, wordNgrams=2, bucket=200000, loss='ova')
# model.test("C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.valid.new")
model.predict("Which baking dish is best to bake a banana bread ?", k=-1, threshold=0.5)

(('__label__baking', '__label__bread', '__label__equipment'),
 array([0.99737835, 0.98718882, 0.66542059]))

In [43]:
pd.read_table("C:/ZhangLI/Codes/DataSet/cooking.stackexchange/cooking.valid.new")

,__label__equipment __label__cast-iron how do i fix a cast iron pot that was heated empty for hours ?
0,__label__oven how does grill / broil mode in a...
1,__label__sauce __label__indian-cuisine __label...
2,__label__chili-peppers __label__spicy-hot how ...
3,__label__bread what is the secret to baking br...
4,__label__eggs are egg whites generally availab...
...,...
2994,__label__baking __label__substitutions __label...
2995,__label__beef __label__roast-beef roasting sir...
2996,__label__vegetables how to use csa vegetables ...
2997,__label__culinary-uses __label__seeds what are...
